<a href="https://colab.research.google.com/github/makofanova/IntentCatcher/blob/main/IntentCatcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
!pip install gensim==4.1.2
!pip install tensorflow==2.5
!pip install sklearn
!pip install numpy
!pip install pymorphy2
!pip install pickle
!pip install collections


     |████████████████████████████████| 24.1 MB 1.9 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 454.3 MB 12 kB/s 
     |████████████████████████████████| 462 kB 44.0 MB/s 
     |████████████████████████████████| 4.0 MB 26.7 MB/s 
     |████████████████████████████████| 132 kB 57.4 MB/s 
     |████████████████████████████████| 1.2 MB 43.5 MB/s 
     |████████████████████████████████| 14.8 MB 39.3 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68724 sha256=a79256084ae771e35f6a20de9d7bf524e2d97ab80d0221361f7e05fddaccd113
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
  

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections


In [ ]:
import nltk
nltk.download('stopwords')

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdo

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import gensim
import tensorflow.keras as keras
import nltk
import sklearn
import numpy
import collections
import pymorphy2
import pickle
import gensim.downloader
import os.path
import typing
import numpy as np
from tensorflow.keras.utils import to_categorical

__EMBED_SIZE = 300
__NUM_FILTERS = 512
__NUM_WORDS = 3
__BATCH_SIZE = 32
__NUM_EPOCHS = 15
__keras_model_path = 'keras_model.h5'
__tokenizer_path = 'tokenizer.pickle'
__maxlen_path = 'maxlen.bin'

def fit(requests, category_ids): 
    save_category_ids(category_ids)  

    category_ids = get_consecutive_indices(category_ids)  
    requests, category_ids = sklearn.utils.shuffle(requests, category_ids) 

    sentences = numpy.array([tokenize(request) for request in requests]) 
    counter, maxlen = get_counter_and_maxlen(sentences)
    vocab_sz = len(counter) + 1 
    embedding_weights = get_embedding_weights(counter, vocab_sz)
    X_train = get_X_train(sentences, maxlen)
    y_train = keras.utils.to_categorical(category_ids)
    keras_model = get_model(vocab_sz, maxlen, embedding_weights, classes_count=len(numpy.unique(category_ids)))

    history = keras_model.fit(X_train, y_train, batch_size=__BATCH_SIZE,
                              epochs=__NUM_EPOCHS,
                              callbacks=[keras.callbacks.ModelCheckpoint('keras_model.h5', save_best_only=True)],
                              validation_split=0.2, verbose=0)

    return history

def predict(request: str) -> int:
    if os.path.isfile(__keras_model_path) and os.path.isfile(__tokenizer_path) and os.path.isfile(__maxlen_path):
        keras_model = keras.models.load_model(__keras_model_path) 

        with open(__tokenizer_path, 'rb') as handle:
          tokenizer = pickle.load(handle)  
        sentences = [tokenize(request)]
        X_predict = tokenizer.texts_to_sequences(sentences)
        X_predict = keras.preprocessing.sequence.pad_sequences(X_predict, maxlen=get_maxlen())

        prediction = np.argmax(keras_model.predict(X_predict), axis=-1)
        indices_unique = numpy.unique(get_category_ids()) 

        return indices_unique[prediction[0]]
    else:
        print('Error')

        return -1

def get_consecutive_indices(category_ids):
    consecutive_indices = []
    unique_id = numpy.unique(category_ids).tolist()

    for id in category_ids:
        consecutive_indices.append(unique_id.index(id))

    return consecutive_indices

def save_category_ids(category_ids):
    with open('indices.pickle', 'wb') as f:
        pickle.dump(category_ids, f)

def get_category_ids():
    with open('indices.pickle', 'rb') as f:
        indices = pickle.load(f)

    return indices

def get_embedding_weights(counter, vocab_sz):
    word2vec_model = gensim.downloader.load("word2vec-ruscorpora-300")

    embedding_weights = numpy.zeros(
        (vocab_sz, __EMBED_SIZE)) 
    index = 0
    sorted_counter = counter.most_common()  
    for word in sorted_counter: 
        try:
            embedding_weights[index, :] = word2vec_model[word[0]]
            index += 1
        except KeyError: 
            index += 1
            pass

    return embedding_weights

def get_counter_and_maxlen(sentences):
    counter = collections.Counter()

    maxlen = 0
    for words in sentences:
        if len(words) > maxlen:
            maxlen = len(words)
        for word in words:
            counter[word] += 1

    save_maxlen(maxlen)

    return counter, maxlen

def save_maxlen(maxlen):
    f = open(__maxlen_path, 'w')
    f.write(str(maxlen))
    f.close()

def get_maxlen():
    f = open(__maxlen_path, 'r')
    maxlen = int(f.read())
    f.close()

    return maxlen

def get_X_train(sentences, maxlen):
    tokenizer = keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(sentences)
    X_train = tokenizer.texts_to_sequences(sentences)  
    X_train = keras.preprocessing.sequence.pad_sequences(X_train,
                                                         maxlen=maxlen)
    save_tokenizer(tokenizer)

    return X_train

def save_tokenizer(tokenizer):
    with open(__tokenizer_path, 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

def tokenize(request):
    morph = pymorphy2.MorphAnalyzer() 
    regex_tokenizer = nltk.tokenize.RegexpTokenizer('[а-яА-ЯЁё]+')
    words = regex_tokenizer.tokenize(request.lower())
    stop_words = set(nltk.corpus.stopwords.words("russian"))
    without_stop_words = [(morph.parse(w)[0]).normal_form 
    for w in words if w not in stop_words and len(w) > 1]
    output = [add_part_of_speech(morph, word) for word in without_stop_words]
    return output

def add_part_of_speech(morph, word):
    p = morph.parse(word)[0]
    word += '_' + str(p.tag.POS)
    return word

def get_model(vocab_sz, maxlen, embedding_weights, classes_count):
    model = keras.models.Sequential()
    model.add(keras.layers.Embedding(vocab_sz, __EMBED_SIZE, input_length=maxlen,
                                     weights=[embedding_weights],
                                     trainable=True))

    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Conv1D(50,
                                  3,
                                  padding='valid',
                                  activation='relu',
                                  strides=1))
    model.add(keras.layers.GlobalMaxPooling1D())
    model.add(keras.layers.Dense(250))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Dense(classes_count, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
import json
requests = []
category_ids = []
with open('talk_template.json', 'r', encoding='utf-8') as f:
    answers = json.load(f)
for i in range(len(answers)):
  for j in answers[i]['action']:
    requests.append(j)
    category_ids.append(i)
print(category_ids)
fit(requests, category_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


[==================================================] 100.0% 198.8/198.8MB downloaded


In [ ]:
%time 
id = predict('какое расписание завтра')
print(id)
answers[id]['action'][0]

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs
85


'какое расписание сегодня'

In [ ]:
req = ['Привет', 'Пока', 'Какая погода завтра']
cat = [1, 2, 3]